In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

iris_df = sns.load_dataset("iris")
titanic_df = sns.load_dataset("titanic")
ts_df = sns.load_dataset("dowjones")
iris_df

## Medidas de Tendencia Central

In [ ]:
print(f"Promedio de Ancho de Petalo {iris_df['sepal_width'].mean()}")
print(f"Mediana de Largo de Petalo {iris_df['sepal_length'].median()}")

In [ ]:
print(f"Moda de Especies: ")
iris_df["species"].mode()

In [ ]:
p25 = iris_df["sepal_width"].quantile(q=0.25)
p50 = iris_df["sepal_width"].quantile(q=0.50)
p75 = iris_df["sepal_width"].quantile(q=0.75)
iris_df["sepal_width"].median(), p25, p50, p75

## Medidas de Dispersión

In [ ]:
# Mostrar error en caso de no usar numeric_only
iris_df.var(numeric_only=True, ddof=0)

In [ ]:
iris_df.var(numeric_only=True, ddof=1)

In [ ]:
iris_df.std(numeric_only=True, ddof=1)

In [ ]:
# Mostrar cómo se construye esta función...
def calculate_quantiles(column):
    quantiles = iris_df.quantile([0.25, 0.75], numeric_only=True)
    iqr_sl = quantiles.loc[0.75, column] - quantiles.loc[0.25, column]
    return iqr_sl


calculate_quantiles("sepal_length")
calculate_quantiles("petal_width")

In [ ]:
iris_df.skew(numeric_only=True)

> Cuando hacemos visualizaciones tenemos 2 formas de hacerlas: 
* Usar Pandas directamente y/o Matplotlib: Buena documentación, gráficos bonitos.
* Usar Seaborn: Documentación Complicada, fácil para descubrir insights.

## Visualizaciones: Pandas

### Histogramas

In [ ]:
iris_df.hist(figsize=(20, 6), bins=30, edgecolor="black", grid=False)
plt.tight_layout()

In [ ]:
iris_df.plot(
    kind="hist", alpha=0.5, bins=30, figsize=(20, 6), edgecolor="black"
)
# Notar que este genera todos los histogramas super puestos...

### Barplots

In [ ]:
titanic_df["sex"].value_counts().plot(
    kind="bar",
    figsize=(5, 6),
    title="Número de Pasajeros por Sexo...",
    edgecolor="black",
)

> Para este tipo de gráficas se requiere entender el concepto de agrupación...

In [ ]:
## Gráficar varias dimensiones
titanic_df.groupby("pclass")[["age", "fare"]].mean().plot(
    kind="bar", edgecolor="black", title="Edad y Tarifa por cada Clase"
)

In [ ]:
iris_df.groupby("species").mean().plot(kind="bar", edgecolor="black")

## Boxplots

In [ ]:
iris_df.drop(columns="species").plot(kind="box")

### Puntos

In [ ]:
iris_df.plot(
    x="petal_length",
    y="petal_width",
    kind="scatter",
    title="Largo de Pétalo vs Ancho de Pétalo",
    xlabel="Largo",
    ylabel="Ancho",
)

In [ ]:
ts_df.plot(x="Date", y="Price", title="Evolución del Dow Jones")

In [ ]:
from scipy.stats import norm

ts_df["AA"] = ts_df["Price"] + norm.rvs(size=649) * 55 + 1000
ts_df["BB"] = -norm.rvs(size=649) * 55

ts_df.set_index("Date").plot(title="Comparación distintas Tendencias")

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.subplot_mosaic(
    """AAA
       BCC"""
)

iris_df.drop(columns="species").plot(
    kind="box", ax=ax["C"], title="Distribución de Datos por Variable"
)
iris_df.plot(
    x="petal_length",
    y="petal_width",
    kind="scatter",
    title="Largo de Pétalo vs Ancho de Pétalo",
    xlabel="Largo",
    ylabel="Ancho",
    ax=ax["B"],
)
iris_df.groupby("species").mean().plot(
    kind="bar",
    edgecolor="black",
    ax=ax["A"],
    rot=0,
    title="Valores promedio por Especie",
)
plt.suptitle("Súper Título", fontsize=20)

## Seaborn

In [ ]:
sns.catplot(kind="count", data=titanic_df, y="embark_town")
plt.title("Pasajeros por Puerto")
sns.catplot(kind="count", data=titanic_df, x="pclass")
plt.title("Pasajeros por Clase")

> En el caso de querer entender los colores de Seaborn ver [acá](https://seaborn.pydata.org/tutorial/color_palettes.html#palette-tutorial)

In [ ]:
# Agrupar por ambos casos
sns.catplot(
    kind="count", y="sex", hue="pclass", data=titanic_df, palette="Set1"
)

In [ ]:
## Barplot
import numpy as np

sns.catplot(
    kind="bar",
    x="sex",
    y="fare",
    data=titanic_df,
    estimator=np.median,
    errorbar=None,
    hue="embark_town",
    col="pclass",
)

### Histogramas

In [ ]:
sns.displot(x="fare", data=titanic_df, kind="kde", height=4, aspect=6)

In [ ]:
sns.displot(x="fare", data=titanic_df, height=4, aspect=6)

In [ ]:
sns.displot(x="fare", data=titanic_df, kde=True, height=4, aspect=6)

## Lineplots

In [ ]:
sns.relplot(
    kind="scatter", x="age", y="fare", data=titanic_df, hue="pclass"
)

In [ ]:
sns.relplot(x="Date", y="Price", data=ts_df, kind="line")
plt.title("Precio del Dow Jones")

In [ ]:
new_data = ts_df.melt(
    id_vars=["Date"],
    value_vars=["AA", "BB"],
    var_name="Tipo",
    value_name="stock_price",
)
new_data

In [ ]:
sns.relplot(
    kind="line", y="stock_price", x="Date", data=new_data, hue="Tipo"
)

In [ ]:
sns.relplot(
    kind="scatter",
    x="age",
    y="fare",
    data=titanic_df,
    col="alone",
    hue="survived",
)

In [ ]:
# Esto es un insight... Los hombres solos sobrevivieron muchísimo más...
sns.catplot(
    kind="count", x="sex", hue="survived", data=titanic_df, col="alone"
)

> El contra de estos gráficos en Seaborn es que no permiten agregarse a un `Layout Mosaic`.